In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('round1_iflyad_train.txt',sep='\t')

### 《广告CTR分析报告》
* **任务**：分析广告CTR在各维度的表现情况


* **方式**：
* 1、从用户信息、媒体信息、广告信息等三个角度对广告CTR进行**单维度/双维度**分析。
* 2、利用L1正则与树模型特征重要性进行维度选择。


* **结果**：
* 形成数据分析报告.ppt

[![ymdOfK.png](https://s3.ax1x.com/2021/02/02/ymdOfK.png)](https://imgchr.com/i/ymdOfK)

* 数据长怎样

In [3]:
df.head()

,instance_id,time,city,province,user_tags,carrier,devtype,make,model,nnt,...,creative_width,creative_height,creative_is_jump,creative_is_download,creative_is_js,creative_is_voicead,creative_has_deeplink,app_paid,advert_name,click
0,86294719979897807,2190219034,137103102105100,137103102100100,NaN,1,2,HUAWEI,HUAWEI-CAZ-AL10,1,...,1280,720,True,False,False,False,False,False,B4734117F35EE97F,0
1,2699289844928136052,2190221070,137105101100100,137105101100100,"2100191,2100078,3001825,,3001781,3001791,30017...",3,2,Xiaomi,Redmi Note 4,1,...,960,640,True,False,False,False,False,False,B4734117F35EE97F,0
2,3117527168445845752,2190219793,137103104111100,137103104100100,NaN,3,2,OPPO,OPPO+R11s,1,...,960,640,True,False,False,False,False,False,E257895F74792E81,0
3,3398484891050993371,2190221704,137103102113100,137103102100100,"2100098,gd_2100000,3001791,3001795,3002193,300...",0,2,NaN,OPPO A57,1,...,1280,720,True,False,False,False,False,False,0A421D7B11EABFC5,0
4,2035477570591176488,2190220024,137103102109100,137103102100100,NaN,1,2,Apple,iPhone 7,3,...,960,640,True,False,False,False,False,False,B4734117F35EE97F,0


In [4]:
df.shape

(1001650, 35)

#### 探索分析

In [14]:
# 点击率
df.groupby('province',as_index=False)['click'].mean().sort_values('click',ascending=False)

,province,click
6,137102101100100,0.240456
8,137102103100100,0.238303
30,137106104100100,0.237465
4,137101104100100,0.237214
2,137101102100100,0.235897
28,137106102100100,0.232011
29,137106103100100,0.230290
22,137105101100100,0.229753
15,137103107100100,0.223913
23,137105102100100,0.223302


In [6]:
# 点击量，曝光量，PV（页面浏览量）

In [7]:
df.groupby('province',as_index=False)['click'].count()

,province,click
0,0,17640
1,137101101100100,15183
2,137101102100100,11522
3,137101103100100,57933
4,137101104100100,37755
5,137101105100100,20297
6,137102101100100,34709
7,137102102100100,25242
8,137102103100100,25732
9,137103101100100,12691


In [8]:
def toprate_function(data,col):
    '''
    data：代表数据集
    col：代表特征名
    '''
    data_ctr = data.groupby(col,as_index=False)['click'].mean().sort_values('click',ascending=False)
    data_pv = data.groupby(col,as_index=False)['click'].count()
    data_c = pd.merge(data_ctr,data_pv,how='left',on=col)
    data_c.columns = ['province','ctr','pv']
    
    return data_c

In [9]:
df_c = toprate_function(df,'province')

In [10]:
df_c.sort_values('ctr',ascending=False)

,province,ctr,pv
0,137102101100100,0.240456,34709
1,137102103100100,0.238303,25732
2,137106104100100,0.237465,4607
3,137101104100100,0.237214,37755
4,137101102100100,0.235897,11522
5,137106102100100,0.232011,14370
6,137106103100100,0.230290,3209
7,137105101100100,0.229753,19299
8,137103107100100,0.223913,76811
9,137105102100100,0.223302,51849


In [11]:
df_c[df_c['pv']>df_c['pv'].mean()].iloc[:10]

,province,ctr,pv
0,137102101100100,0.240456,34709
3,137101104100100,0.237214,37755
8,137103107100100,0.223913,76811
9,137105102100100,0.223302,51849
11,137103102100100,0.214860,66178
13,137104101100100,0.210209,63023
17,137106101100100,0.200038,31789
18,137101103100100,0.199075,57933
20,137104103100100,0.197872,35907
24,137104102100100,0.182936,34575


In [12]:
# islide

#### 双维度

In [13]:
df_pc = df.groupby(['province','city'],as_index=False)['click'].mean()

In [15]:
df.groupby(['province','city'],as_index=False)['click'].mean()

,province,city,click
0,0,0,0.191383
1,137101101100100,137101101100100,0.169071
2,137101102100100,137101102100100,0.235897
3,137101103100100,137101103101100,0.206914
4,137101103100100,137101103102100,0.220071
...,...,...,...
328,137106105100100,137106105114100,0.254355
329,137106105100100,137106105115100,0.200680
330,137107101100100,137107101100100,0.054878
331,137107102100100,137107102100100,0.094077
